# 1. Model-data calibration
In this module, we are going to be focusing on coding up model-data calibration using Bayesian and frequentist approaches.

In this particular Python script, we are working on the **frequentist** approach, for 3 model examples:
1.    A toy example (fitting an exponential model to some toy data).
2.    Michaelis-Menten kinetics for chemical reactions (as in [Monsalve-Bravo *et al.* 2022 Sci. Adv.](https://www.science.org/doi/10.1126/sciadv.abm5952)).
3.    Logistic growth model for coral reef recovery (as in [Simpson *et al.* 2022 J. Theor. Biol.](https://www.sciencedirect.com/science/article/pii/S0022519321004185)).

### 1.1 Preliminary code to run!
We first need to import the required libraries for Python:
*  *matplotlib.pyplot* (for generating plots)
*  *numpy* (for linear algebra routines)
*  *lmfit* (for least-squares regression routines)
*  *math* (for mathematical functions)

In [ ]:
## Loading Python libraries
import matplotlib.pyplot as plt
import numpy as np
import math
!pip install lmfit
# Note: This "lmfit" installation step only needs to be done once
from lmfit import Model

### 1.2 Which data?

We are going to consider a total of (up to) five datasets here:

1. Data for a toy model.
2. Three datasets for Michaelis-Menten kinetics (low concentration data, high concentration data, and combined low and high concentration data).
3. Data for coral reef recovery.

The key thing is that we choose below which dataset we are analysing at any given time, by placing this data in the arrays **x_obs** and **y_obs**.

We then do a preliminary plot of the data (before fitting it to the model).

In [ ]:
## 1. Data for a toy model
x_obs = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
y_obs = np.array([0.2, 5, 9, 12, 19, 20, 30, 38, 52, 80, 140])

# ## 2.1 Data for Michaelis-Menten kinetics (high concentration data)
# x_obs = np.array([1000, 1500, 2000, 2500, 3000])
# y_obs = np.array([436.0465, 518.3769, 492.7244, 412.1804, 470.0788])

# ## 2.2 Data for Michaelis-Menten kinetics (low concentration data)
# x_obs = np.array([10, 20, 30, 40, 50])
# y_obs = np.array([31.9149, 29.0095, 96.5600, 88.4801, 199.3002])

# ## 2.3 Data for Michaelis-Menten kinetics (high and low concentration data)
# x_obs = np.array([10, 20, 30, 40, 50, 1000, 1500, 2000, 2500, 3000])
# y_obs = np.array([31.9149, 29.0095, 96.5600, 88.4801, 199.3002, \
#                 436.0465, 518.3769, 492.7244, 412.1804, 470.0788])

# ## 3. Data for coral reef recovery
# x_obs = np.array([0,800,1100,1500,1900,2200,2600,2900,3200,3600,4000])
# y_obs = np.array([2,4,8,22,39,59,68,69,74,82,81])

## Plot the chosen dataset
plt.plot(x_obs, y_obs, 'o')
plt.show()


### 1.3 Which model?

We are going to consider three different models:

#### Toy exponential model for fitting to toy data

$y = a e^{b x}$, where: $x$ is model input, $y$ is model output, and there are 2 parameters to estimate: $a$ and $b$.

#### Michaelis-Menten kinetics

$\nu = \dfrac{k_{cat} [E_T] [S]}{K_M + S}$, where: $[S]$ is substrate concentration (model input), $\nu$ is reaction rate (model output), and there are 3 parameters to estimate: $[E_T]$, $k_{cat}$ and $K_M$.

#### Logistic model for coral reef recovery

$\dfrac{\mathrm{d}C(t)}{\mathrm{d}t} = rC(t) \left(1 - \dfrac{C(t)}{K} \right), \,\, C(0) = C_0$, where $t$ is time since coral reef recovery began (model input), $C(t)$ is coral reef cover at time $t$ (model output), and there are 3 parameters to estimate: $C_0$, $r$ and $K$.

In [ ]:
## Toy exponential model
def y_model_function(x, a, b):
   y_model = a*np.exp(b*x)
   return y_model

# ## Michaelis-Menten kinetics model
# def y_model_function(S, E_T, k_cat, K_M):
#    nu = k_cat*E_T*S/(K_M+S)
#    return nu

# ## Coral reef recovery model
# def y_model_function(T, C_0, K, r):
#     vector_of_C_at_times_T = np.zeros(len(T))
#     for obs_number in range(len(T)):
#         # For each observation, re-run the model
#         if T[obs_number] == 0:
#             vector_of_C_at_times_T[obs_number] = C_0
#             # No need to run the model if we are looking at an observation at t = 0!
#         else:      
#             approx_dt = 1 # (Approximate) timestep of model
#             num_t_values = round(T[obs_number]/approx_dt) # Number of timesteps in the model.
#             t = np.linspace(0, T[obs_number], num=num_t_values+1) # Set up the vector for time t.
#             dt = t[1]-t[0] # "Delta t", i.e. how spaced apart each time value t actually is.
#             C = np.zeros(num_t_values+1)
#             C[0]=C_0
#             for n in range(num_t_values): # Running the ODE model!
#                 C[n+1]=C[n] + dt * r * C[n] * (1 - C[n]/K)
#             vector_of_C_at_times_T[obs_number] = C[-1]
#     return vector_of_C_at_times_T
        

### 1.4  Fit the model to the data!

Now that we have defined the model, and the data, we can use in-built packages to estimate the *best-fit* model parameters $\theta_{\mathrm{MLE}}$ from the data. Here we will use *lmfit*.

In [ ]:
model_structure = Model(y_model_function)
# Build the model (a necesssary code step for "lmfit", this code step may be unnecessary in other programs)

params = model_structure.make_params(a=1,b=1)
# Input some initial guesses for the parameters we are estimating!

fitted_model = model_structure.fit(y_obs, params, x=x_obs)
print(fitted_model.fit_report())


### 1.5 Plot the model-data fit.

In [ ]:
plt.plot(x_obs, y_obs, 'ok')

x_model = np.linspace(min(x_obs),max(x_obs),101)
y_model = fitted_model.eval(x=x_model)

plt.plot(x_model, y_model, 'r')
plt.show()



# 2. Visualisation

In this module, we are going to focus on visualising our calibration outputs.

We have already seen in Section 1.5 how to plot the model-data fit of the MLE.

We will do five types of visualisation in this module (2.1-2.5), but only need frequentist model-data calibration outputs for two of them: **2.1** Confidence interval predictions, and **2.4** Extract best-fit parameter estimates (for comparison to Bayesian inference parameter estimates).

### 2.1 Confidence interval predictions

Luckily for us, the in-built package we have used here for frequentist model-data calibration can do the hard work of calculating one type of confidence interval for us - the confidence interval for the *uncertainty in the best-fit predictions*.

If you've run the code in Section 1.5, the code block below is the only new code that is needed.

**Note \#1:** The value assigned to the function argument 'sigma' determines whether the confidence interval is $\pm 1 \sigma$, $\pm 2 \sigma$ or $\pm 3 \sigma$. Simply change this value to 1, 2 or 3 to switch it! (You can also choose any other multiple of sigma if you so fancy.)

**Note \#2:** The code block below is set up for the toy exponential model example: you'll need to change 'x=x_model' to 'S=x_model' or 'T=x_model' to make it work for the Michaelis-Menten kinetics or coral reef recovery models, respectively.

**Note \#3:** You will likely get something looking quite awful for anything more complicated than the toy exponential model example... Check the uncertainty in parameter estimates obtained in Section 1.4 for a hint at why this might be the case!


In [ ]:
# Calculate the confidence interval for the uncertainty in the best-fit predictions.
# Set sigma = 2 for a 95% confidence interval:
y_model_CI = fitted_model.eval_uncertainty(x=x_model, sigma=2)

plt.plot(x_obs, y_obs, 'ok')
plt.plot(x_model, y_model, 'r')
plt.fill_between(x_model,
                y_model+y_model_CI,
                y_model-y_model_CI,
                color='pink',
                label='uncertainty band of fit')
plt.show()

### 2.4 Extract best-fit parameter estimates (for comparison to Bayesian inference parameter estimates)

The below code extracts parameter values (set up here for the toy exponential model example) for comparison to the Bayesian inference parameter estimates - see Section 2.4 of that Python script!

We are also here calculating the *root-mean-square error* via

\begin{equation*}
\mbox{RMSE} = \sqrt{\dfrac{ \displaystyle\sum_{k=1}^{N_{\mathrm{obs}}} \left( y_{\mathrm{obs},k} - y_{\mathrm{model}} \left(\mathbf{x}_{\mathrm{obs},k}, \mathbf{\theta} \right) \right)^2 }{N_{\mathrm{obs}} - N_{\theta}} } \end{equation*}

as this number is loosely analogous to the standard deviation $\sigma$ that Bayesian inference will also estimate. (Note that sometimes, when RMSE is calculated, $N_{\theta}$ is dropped off from this formula.)



In [ ]:
a_MLE = fitted_model.params['a'].value
b_MLE = fitted_model.params['b'].value
residuals = fitted_model.residual
RMSE = math.sqrt(sum(np.square(residuals))/(len(residuals)-2)) # 2 here because N_theta=2 for the toy exponential model example

print(f'a_MLE=',a_MLE)
print(f'b_MLE=',b_MLE)
print(f'RMSE=',RMSE)


# 3. Analysis of model slopppiness

In this module we are going to focus on the task of calculating the eigenparameters (parameter combinations) of the model, ordered from stiffest (most sensitive to model-data fit) to sloppiest (least sensitive to model-data fit).

For frequentist model-data calibrations, we will discuss two sensitivity matrices: the Hessian matrix **H** and the Levenberg-Marquardt Hessian matrix **L**.

### 3.1 Hessian matrix

Let's start by doing analysis of sloppiness using the *Hessian matrix* **H** as the sensitivity matrix.

The code below is set up for the toy exponential model example, but can be changed to other examples by:
* Changing the values of theta_MLE, and
* Changing the parameter names, and
* Changing the number of theta parameters passed to y_model inside the loglikelihood function definition.

The code block ends by printing out the eigenparameters, ordered from stiffest to sloppiest.

In [ ]:
theta_MLE = np.array([[fitted_model.params['a'].value], \
                       [fitted_model.params['b'].value]])   # theta_MLE for the toy exponential model

#theta_MLE = np.array([[fitted_model.params['E_T'].value], \
#                      [fitted_model.params['k_cat'].value], \
#                      [fitted_model.params['K_M'].value]])   # theta_MLE for the Michaelis-Menten kinetics models

# theta_MLE = np.array([[fitted_model.params['C_0'].value], \
#                       [fitted_model.params['K'].value], \
#                       [fitted_model.params['r'].value]])     # theta_MLE for the coral reef recovery model

Parameter_names = ['a','b']              # Ordered parameter names for the toy exponential model
#Parameter_names = ['E_T','k_cat','K_M'] # Ordered parameter names for the Michaelis-Menten kinetics models
#Parameter_names = ['C_0','K','r']       # Ordered parameter names for the coral reef recovery model

# Define loglikelihood function for the present model.
def loglikelihood_function(x_obs, y_obs, theta):
  params = theta
  sigma = 1 # It doesn't matter too much what the value of sigma is!
  y_model = y_model_function(x_obs, theta[0], theta[1])
            # You'll need to change this when you change the number of parameters in your model!
  N_obs = len(y_obs)
  Loglikelihood = -0.5*N_obs*np.log(2*math.pi) \
                  - 0.5*N_obs*np.log( np.square(sigma)) \
                  - 0.5 * np.sum( np.square((y_obs-y_model)/sigma) )
  return Loglikelihood

# Define common things needed to calculate H
logL_MLE = loglikelihood_function(x_obs,y_obs,theta_MLE)
delta = 1e-4
N_theta = len(theta_MLE)
H = np.zeros((N_theta,N_theta))

# First, calculate the diagonal elements of Hessian matrix H
for i in range(N_theta):
    theta_up = np.copy(theta_MLE)
    theta_up[i]=theta_MLE[i]*(1 + delta)
    logL_up = loglikelihood_function(x_obs,y_obs,theta_up)
    theta_down = np.copy(theta_MLE)
    theta_down[i]=theta_MLE[i]*(1 - delta)
    logL_down = loglikelihood_function(x_obs,y_obs,theta_down)
    H[i,i]= -1/np.square(delta) * (logL_up - 2*logL_MLE + logL_down)

# Second, calculate the off-diagonal elements of Hessian matrix H
for i in range(N_theta):
    for j in range(i+1,N_theta):
        theta_up_up = np.copy(theta_MLE)
        theta_up_up[i]=theta_MLE[i]*(1 + delta/2)
        theta_up_up[j]=theta_MLE[j]*(1 + delta/2)
        logL_up_up = loglikelihood_function(x_obs,y_obs,theta_up_up)
        theta_up_down = np.copy(theta_MLE)
        theta_up_down[i]=theta_MLE[i]*(1 + delta/2)
        theta_up_down[j]=theta_MLE[j]*(1 - delta/2)
        logL_up_down = loglikelihood_function(x_obs,y_obs,theta_up_down)
        theta_down_up = np.copy(theta_MLE)
        theta_down_up[i]=theta_MLE[i]*(1 - delta/2)
        theta_down_up[j]=theta_MLE[j]*(1 + delta/2)
        logL_down_up = loglikelihood_function(x_obs,y_obs,theta_down_up)
        theta_down_down = np.copy(theta_MLE)
        theta_down_down[i]=theta_MLE[i]*(1 - delta/2)
        theta_down_down[j]=theta_MLE[j]*(1 - delta/2)
        logL_down_down = loglikelihood_function(x_obs,y_obs,theta_down_down)
        H[i,j] = -1/np.square(delta) * (logL_up_up - logL_up_down - logL_down_up + logL_down_down)
        H[j,i] = np.copy(H[i,j])

lamda, v = np.linalg.eig(H)
# Perform eigendecomposition on the Hessian matrix H!

reordered_lamda_elements = lamda.argsort()[::-1]
lamda = lamda[reordered_lamda_elements]
v = v[:,reordered_lamda_elements]
# Reorder eigenvalues from largest to smallest, and reorder the eigenvectors accordingly

lamda = lamda/max(lamda)
# Rescale the eigenvalues.

for n in range(N_theta):
    if abs(min(v[:,n])) > max(v[:,n]):
        v[:,n]=v[:,n]/min(v[:,n])
    else:
        v[:,n]=v[:,n]/max(v[:,n])
# Rescale each eigenvector so that the leading parameter within each eigenvector has index of +1

for j in range(len(theta_MLE)):
    Eigenparameter = Parameter_names[0] + '^' + str(round(v[0,j],2))
    for i in range(1,len(theta_MLE)):
        Eigenparameter = Eigenparameter + ' x ' + Parameter_names[i] + '^' + str(round(v[i,j],2))
    print(f'Eigenparameter',j+1,'is',Eigenparameter,'corresponding to lambda_j/lambda_1=',round(lamda[j],3))
# Report the eigenparameters!
# Note we don't bother here to remove parameters with indices between -0.2 and +0.2, I leave that you to interpret correctly!



### 3.2 Levenberg-Marquardt Hessian matrix L

We can alternatively use the Levenberg-Marquardt Hessian matrix **L** matrix as the sensitivity matrix: we should get the same result, but it may be slightly faster computationally. (However, for the small models we are analysing in this course, the only time you might notice *any* difference in computation time is for the coral model!)

Like Section 3.1, the code below is set up for the toy exponential model example, but can be changed to other examples by:
* Changing the values of theta_MLE, and
* Changing the parameter names, and
* Changing the number of theta parameters passed to y_model inside the $r$ function definition.

The code block ends by printing out the eigenparameters, ordered from stiffest to sloppiest.

In [ ]:
theta_MLE = np.array([[fitted_model.params['a'].value], \
                       [fitted_model.params['b'].value]])   # theta_MLE for the toy exponential model

#theta_MLE = np.array([[fitted_model.params['E_T'].value], \
#                      [fitted_model.params['k_cat'].value], \
#                      [fitted_model.params['K_M'].value]])   # theta_MLE for the Michaelis-Menten kinetics models

# theta_MLE = np.array([[fitted_model.params['C_0'].value], \
#                       [fitted_model.params['K'].value], \
#                       [fitted_model.params['r'].value]])     # theta_MLE for the coral reef recovery model

Parameter_names = ['a','b']              # Ordered parameter names for the toy exponential model
#Parameter_names = ['E_T','k_cat','K_M'] # Ordered parameter names for the Michaelis-Menten kinetics models
#Parameter_names = ['C_0','K','r']       # Ordered parameter names for the coral reef recovery model

def r_function(x_obs, y_obs, k, theta):
  sigma = 1 # Again it doesn't matter too much what the value of sigma is!
  y_model_k = y_model_function([x_obs[k]], theta[0], theta[1])
    # You'll need to change this when you change the number of parameters in your model!
  r_k_theta = (y_obs[k]-y_model_k)/sigma
  return r_k_theta

# Define common things needed to calculate L
delta = 1e-4
N_obs = len(y_obs)
N_theta = len(theta_MLE)
dr_dlogtheta = np.zeros((N_obs,N_theta))
L = np.zeros((N_theta,N_theta))

# First, calculate all required derivatives d(r_k)/d(log theta_i)
for i in range(N_theta):
    theta_up = np.copy(theta_MLE)
    theta_up[i] = theta_MLE[i]*(1 + delta/2)
    theta_down = np.copy(theta_MLE)
    theta_down[i] = theta_MLE[i]*(1 - delta/2)
    for k in range(N_obs):
        dr_dlogtheta[k,i] = (r_function(x_obs,y_obs,k,theta_up) \
                             - r_function(x_obs,y_obs,k,theta_down)) / delta

# Second, calculate all elements of Levenberg-Marquardt Hessian matrix L
for i in range(N_theta):
    for j in range(N_theta):
        for k in range(N_obs):
            L[i,j] = L[i,j] + dr_dlogtheta[k,i]*dr_dlogtheta[k,j]
            
lamda, v = np.linalg.eig(L)
# Perform eigendecomposition on the Levenberg-Marquardt Hessian matrix L!

reordered_lamda_elements = lamda.argsort()[::-1]
lamda = lamda[reordered_lamda_elements]
v = v[:,reordered_lamda_elements]
# Reorder eigenvalues from largest to smallest, and reorder the eigenvectors accordingly

lamda = lamda/max(lamda)
# Rescale the eigenvalues.

for n in range(N_theta):
    if abs(min(v[:,n])) > max(v[:,n]):
        v[:,n]=v[:,n]/min(v[:,n])
    else:
        v[:,n]=v[:,n]/max(v[:,n])
# Rescale each eigenvector so that the leading parameter within each eigenvector has index of +1

for j in range(len(theta_MLE)):
    Eigenparameter = Parameter_names[0] + '^' + str(round(v[0,j],2))
    for i in range(1,len(theta_MLE)):
        Eigenparameter = Eigenparameter + ' x ' + Parameter_names[i] + '^' + str(round(v[i,j],2))
    print(f'Eigenparameter',j+1,'is',Eigenparameter,'corresponding to lambda_j/lambda_1=',round(lamda[j],3))
# Report the eigenparameters!
# Note we don't bother here to remove parameters with indices between -0.2 and +0.2, I leave that you to interpret correctly! 
